In [2]:
import os

In [3]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str

In [7]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [9]:
import pandas as pd 
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import numpy as np 
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from CustomerChurn import logger

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred, proba):
        accuracy = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred)
        recall = recall_score(actual, pred)
        auc = roc_auc_score(actual, proba)
        return accuracy, f1, recall, auc
    
    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]  

        y_pred = model.predict(X_test)

        # Handle probability output
        try:
            y_proba = model.predict_proba(X_test)[:, 1]
        except AttributeError:
            y_proba = y_pred  # Fallback if model has no predict_proba

        # Calculate metrics
        accuracy, f1, recall, auc = self.eval_metrics(y_test, y_pred, y_proba)

        # Save metrics to JSON
        scores = {
            "accuracy": accuracy,
            "f1_score": f1,
            "recall_score": recall,
            "auc_score": auc
        }
        save_json(path=Path(self.config.metric_file_name), data=scores)
        
        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap=plt.cm.Blues)
        plt.title("Confusion Matrix")
        plt.savefig(os.path.join(self.config.root_dir, "confusion_matrix.png"))
        plt.close()

        # ROC Curve
        from sklearn.metrics import RocCurveDisplay
        RocCurveDisplay.from_predictions(y_test, y_proba)
        plt.title("ROC Curve")
        plt.savefig(os.path.join(self.config.root_dir, "roc_curve.png"))
        plt.close()

        logger.info("Evaluation metrics and visualizations saved successfully.")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2025-07-25 00:14:52,039] 32 CustomerChurnLogger - common - INFO - yaml file: config\config.yaml loaded successfully
[2025-07-25 00:14:52,042] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2025-07-25 00:14:52,045] 32 CustomerChurnLogger - common - INFO - yaml file: schema.yaml loaded successfully
[2025-07-25 00:14:52,047] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts
[2025-07-25 00:14:52,048] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts/model_evaluation
[2025-07-25 00:14:52,867] 68 CustomerChurnLogger - common - INFO - json file saved at: artifacts\model_evaluation\metrics.json
[2025-07-25 00:14:53,550] 54 CustomerChurnLogger - 741855899 - INFO - Evaluation metrics and visualizations saved successfully.
